In [1]:
!pip install openai tiktoken

# 通过Prompt进行文本改写

In [4]:
from openai import OpenAI
import os

client = OpenAI(
   base_url = os.getenv("OPENAI_API_BASE")
)


def make_text_short(text):
    messages = []
    messages.append({"role": "system", "content": "你是一个用来将文本改写得短的AI助手，用户输入一段文本，你给出一段意思相同，但是短小精悍的结果"})
    messages.append({"role": "user", "content": text})
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=2048,
        presence_penalty=0,
        frequency_penalty=2,
        n=3,
    )
    return response

long_text = """
在这个快节奏的现代社会中，我们每个人都面临着各种各样的挑战和困难。
在这些挑战和困难中，有些是由外部因素引起的，例如经济萧条、全球变暖和自然灾害等。
还有一些是由内部因素引起的，例如情感问题、健康问题和自我怀疑等。
面对这些挑战和困难，我们需要采取积极的态度和行动来克服它们。
这意味着我们必须具备坚韧不拔的意志和创造性思维，以及寻求外部支持的能力。
只有这样，我们才能真正地实现自己的潜力并取得成功。
"""
short_version = make_text_short(long_text)

index = 1
for choice in short_version.choices:
    print(f"version {index}: {choice.message.content}")
    index += 1

version 1: 现代社会中，我们面对各种挑战，包括外部和内部因素。要克服困难，需要积极态度、创造性思维和外部支持。这样才能实现成功。
version 2: 现代社会中，每个人都面临挑战和困难，有外部因素如经济问题、全球变暖，也有内部因素如情感、健康问题。
要克服这些困难，需要积极态度和行动力，并发挥坚韧意志、创造性思维并寻求外部支持。
只有这样才能实现潜力并取得成功。
version 3: 现代社会中，我们都面临挑战：外部如经济问题、全球变暖；内部如情感、健康困扰。要克服这些，需积极态度和坚韧意志，寻求支持并发挥创造力。只有如此，才能实现成功和潜力。


# 通过Logit_bias过滤关键词

In [5]:
import tiktoken

# encoding = tiktoken.get_encoding('p50k_base')
encoding = tiktoken.get_encoding('cl100k_base')
token_ids = encoding.encode("灾害")

print(token_ids)

bias_map = {}
for token_id in token_ids:
    bias_map[token_id] = -100
    # bias_map[token_id] = -1

def make_text_short(text):
    messages = []
    messages.append({"role":"system", "content":"你是一个用来将文本改写得短的AI助手，用户输入一段文本，你给出一段意思相同，但是短小精悍的结果"})
    messages.append({"role": "user", "content": text})
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
        max_tokens=2048,
        presence_penalty=0,
        frequency_penalty=2,
        n=3,
        logit_bias=bias_map
    )
    return response

short_version = make_text_short(long_text)

index = 1
for choice in short_version.choices:
    print(f"version {index}: {choice.message.content}")
    index += 1


[66285, 122, 8676, 111]
version 1: 现代社会充满挑战，外部因素如经济问题和自然环境，内部因素如情感和健康困扰都在考验着我们。要战胜这些困难示坚毅、创新思维，并寻求支持。只有这样，才能实现梦惯并取得成功。
version 2: 在现代社会，我们都面临挑战。有外部因素如经济问题和自然威胁，也有内部因素如情感困扰和健康隐患。要克服这些困障，需要积极态度、坚强意志、创新思维以及寻求支持。只有如此，才能实现成功并发挥潜力。
version 3: 在现代社会中，我们都面临各种挑战。有外部因素如经济问题和自然环境，也有内部因素如情感和健康。要克服这些困障，需要积极态度、创造性思维和寻求支持。只有这样才能实现潜力并取得成功。


In [6]:
def translate(text):
    messages = []
    messages.append({"role":"system", "content":"你是一个翻译，把用户的话翻译成英文"})
    messages.append({"role": "user", "content": text})
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
        max_tokens=2048,
        n=1,
    )
    return response.choices[0].message.content

chinese = long_text
english = translate(chinese)

num_of_tokens_in_chinese = len(encoding.encode(chinese))
num_of_tokens_in_english = len(encoding.encode(english))
print(english)
print(f"chinese: {num_of_tokens_in_chinese} tokens")
print(f"english: {num_of_tokens_in_english} tokens")

In this fast-paced modern society, each of us faces various challenges and difficulties. Some of these challenges and difficulties are caused by external factors such as economic downturns, global warming, and natural disasters. There are also some that are caused by internal factors such as emotional issues, health problems, and self-doubt.

In the face of these challenges and difficulties, we need to adopt a positive attitude and take action to overcome them. This means that we must have a strong will and creative thinking, as well as the ability to seek external support. Only in this way can we truly realize our potential and achieve success.
chinese: 234 tokens
english: 124 tokens


# 列出所有支持的模型

In [7]:
import pandas as pd

engines  = client.models.list()

data = [{'id': d.id, 'owned_by': d.owned_by, 'created': pd.to_datetime(d.created, unit='s')} for d in engines.data]
df = pd.DataFrame(data)
df = df.sort_values(by='created', ascending=False)
display(df[['id', 'owned_by', 'created']])

,id,owned_by,created
16,gpt-4o-2024-05-13,system,2024-05-10 19:08:52
15,gpt-4o,system,2024-05-10 18:50:49
25,gpt-4-turbo-2024-04-09,system,2024-04-08 18:41:17
24,gpt-4-turbo,system,2024-04-05 23:57:21
22,gpt-4-1106-vision-preview,system,2024-03-26 17:10:33
10,gpt-3.5-turbo-0125,system,2024-01-23 22:19:18
3,gpt-3.5-turbo-ca,system,2024-01-23 22:19:18
4,gpt-4-turbo-ca,system,2024-01-23 22:19:18
5,gpt-4-turbo-preview-ca,system,2024-01-23 22:19:18
21,gpt-4-turbo-preview,system,2024-01-23 19:22:57


# 看看模型维度数量

In [8]:
from openai import OpenAI
import os

client = OpenAI(
    base_url=os.getenv("OPENAI_API_BASE"),
)

def get_embedding(text, engine="text-embedding-ada-002"):
    result = client.embeddings.create(input=[text], model=engine)
    return result.data[0].embedding

text = "让我们来算算Embedding"

embedding = get_embedding(text, engine="text-embedding-ada-002")
print("text-embedding-ada-002: ", len(embedding))

embedding = get_embedding(text, engine="text-embedding-3-large")
print("text-embedding-3-large: ", len(embedding))



text-embedding-ada-002:  1536
text-embedding-3-large:  3072


# Inserting Text接口

In [9]:
prefix = """在这个快节奏的现代社会中，我们每个人都面临着各种各样的挑战和困难。
在这些挑战和困难中，有些是由外部因素引起的，例如经济萧条、全球变暖和自然灾害等。\n"""
# 还有一些是由内部因素引起的，例如情感问题、健康问题和自我怀疑等。
suffix = """\n面对这些挑战和困难，我们需要采取积极的态度和行动来克服它们。
这意味着我们必须具备坚韧不拔的意志和创造性思维，以及寻求外部支持的能力。
只有这样，我们才能真正地实现自己的潜力并取得成功。"""

def insert_text(prefix, suffix):
    response = client.completions.create(
        model = 'gpt-3.5-turbo-instruct',
        prompt = prefix,
        suffix = suffix,
        max_tokens = 1024,
    )
    return response

response = insert_text(prefix, suffix)
print(response.choices[0].text)

还有一些是由内部因素造成的，例如压力、焦虑和自我怀疑等。
无论是什么原因，这些挑战和困难都会给我们带来压力和不安。



In [10]:
prefix = """在这个快节奏的现代社会中，我们每个人都面临着各种各样的挑战和困难。
在这些挑战和困难中，有些是由外部因素引起的，例如经济萧条、全球变暖和自然灾害等。\n"""
# 还有一些是由内部因素引起的，例如情感问题、健康问题和自我怀疑等。
suffix = """面对这些挑战和困难，我们需要采取积极的态度和行动来克服它们。
这意味着我们必须具备坚韧不拔的意志和创造性思维，以及寻求外部支持的能力。
只有这样，我们才能真正地实现自己的潜力并取得成功。"""

response = insert_text(prefix, suffix)
print(response.choices[0].text)

还有一些是由内部因素造成的，例如压力、焦虑和自我怀疑等。
无论是什么原因，这些挑战和困难都会给我们带来压力和不安，甚至可能阻碍我们的成长和发展。

然而，我们必须意识到，挑战和困难是生活中不可避免的一部分。
它们可以帮助我们成长和学习，让我们变得更加坚强和有韧性。
正如一句古老的谚语所说：“磨难是上天赐予我们的最好的老师。”

因此，当我们面对挑战和困难时，我们应该学会如何应对和克服它们。
首先，我们需要保持积极的心态。
这意味着我们要相信自己的能力，相信自己可以克服困难。
我们也可以寻求他人的帮助和支持，与他人分享我们的感受和困难，从而获得更多的建议和支持。

其次，我们需要学会如何应对压力和焦虑。
这可能包括通过运动、冥想或与朋友聊天来放松自己。
我们也可以寻求专业的帮助，如心理咨询师或治疗师，来帮助我们处理压力和焦虑。

最后，我们需要学会如何从挑战和困难中学习和成长。
每次面对挑战和困难，我们都可以从中汲取经验教训，从而变得更加坚强和有韧性。
我们也可以通过反思和总结，找出自己的不足之处，并努力改进自己。

总的来说，


# Moderate接口

In [11]:
def chatgpt(text):
    messages = []
    messages.append( {"role": "system", "content": "You are a useful AI assistant"})
    messages.append( {"role": "user", "content": text})
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
        max_tokens=2048,
        top_p=1,
    )
    message = response.choices[0].message.content
    return message

threaten = "你不听我的我就拿刀砍死你"
print(chatgpt(threaten))

请不要发表任何暴力、威胁或不当言论。如果您有任何问题或需要帮助，请告诉我，我会尽力提供支持和解决方案。感谢您的合作。


In [12]:
import json

threaten = "你不听我的我就拿刀砍死你"

def moderation(text):
    response = client.moderations.create(
        input = text
    )

    return response.results[0].json()


print(moderation(threaten))

{"categories":{"harassment":true,"harassment_threatening":true,"hate":false,"hate_threatening":false,"self_harm":false,"self_harm_instructions":false,"self_harm_intent":false,"sexual":false,"sexual_minors":false,"violence":true,"violence_graphic":false,"hate/threatening":false,"self-harm":false,"sexual/minors":false,"violence/graphic":false,"harassment/threatening":true,"self-harm/intent":false,"self-harm/instructions":false},"category_scores":{"harassment":0.8036670684814453,"harassment_threatening":0.6991801261901855,"hate":0.00020632389350794256,"hate_threatening":0.00012161156337242573,"self_harm":0.02363060973584652,"self_harm_instructions":0.0010441517224535346,"self_harm_intent":0.01668197475373745,"sexual":0.0001084490359062329,"sexual_minors":1.7344339084957028e-6,"violence":0.9991687536239624,"violence_graphic":0.008057454600930214,"hate/threatening":0.00012161156337242573,"self-harm":0.02363060973584652,"sexual/minors":1.7344339084957028e-6,"violence/graphic":0.0080574546009

In [13]:
!pip list

Package                        Version
------------------------------ --------------
aiofiles                       22.1.0
aiosqlite                      0.20.0
altair                         5.3.0
annotated-types                0.6.0
anyio                          4.3.0
argon2-cffi                    23.1.0
argon2-cffi-bindings           21.2.0
arrow                          1.3.0
astroid                        3.1.0
asttokens                      2.4.1
attrs                          23.2.0
autopep8                       2.0.4
Babel                          2.14.0
backoff                        2.2.1
beautifulsoup4                 4.12.3
bleach                         6.1.0
certifi                        2024.2.2
cffi                           1.16.0
charset-normalizer             3.3.2
click                          8.1.7
comm                           0.2.2
contourpy                      1.2.1
cramjam                        2.8.3
cycler                         0.12.1
debugpy        